In [31]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

import mlflow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch_geometric.nn import DynamicEdgeConv, global_max_pool

from pointnet import PointNetCls
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [23]:
import pptk

def view(points):
    v = pptk.viewer(points)
    v.attributes(points)
    v.set(point_size=0.01)
    # selected = points[v.get('selected')]
    return v

def plot_conf_matrix(conf_mtrx, labels, file_path='temp_conf_mtrx.png'):
    df_cm = pd.DataFrame(np.array(conf_mtrx), index=labels, columns=labels)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig(file_path)
    plt.close()

In [9]:
batchsize = 32
blue = lambda x: '\033[94m' + x + '\033[0m'
yellow = lambda x: '\033[93m' + x + '\033[0m'
red = lambda x: '\033[91m' + x + '\033[0m'
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(1024)
train_dataset = ModelNet('../data/modelnet10', '10', True, transform, pre_transform)
test_dataset = ModelNet('../data/modelnet10', '10', False, transform, pre_transform)

class Adapter:
    def __init__(self, pg_dataset):
        self.pg_dataset = pg_dataset

    def __len__(self):
        return len(self.pg_dataset)
    
    def __getitem__(self, idx):
        return self.pg_dataset[idx].pos.numpy(), self.pg_dataset[idx].y.numpy().item() 
    
train_loader = torch.utils.data.DataLoader(Adapter(train_dataset), batch_size=batchsize, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(Adapter(test_dataset), batch_size=batchsize, shuffle=False, num_workers=0)
num2cat = dict(zip(range(10), train_dataset.raw_file_names))

AttributeError: 'ModelNet' object has no attribute 'dataset'

In [ ]:
num_classes = len(num2cat)
num_batch = len(train_dataset)/batchsize

classifier = PointNetCls(k=num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier.to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)    
scheduler = StepLR(optimizer, step_size=1.0, gamma=0.99)
mlflow.log_param('Start', 'From zero')

step = 0
for epoch in range(1):
    train_acc_epoch, test_acc_epoch = [], []
    for i, data in enumerate(train_loader):
        points, labels = data
        points = points.transpose(2, 1)
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        loss = F.nll_loss(pred, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(labels.data).cpu().sum()
        train_acc = correct.item() / float(batchsize)
        print('epoch %d: %d/%d | train loss: %f | train acc: %f' % (epoch+1, i+1, num_batch+1, loss.item(), train_acc))
        train_acc_epoch.append(train_acc)
        
        # log train
        encountered_class_nums = np.unique( np.array(labels.cpu().tolist()))
        prec, recall, f1, _ = precision_recall_fscore_support(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(encountered_class_nums))
        for j, num in enumerate(encountered_class_nums):
            name = num2cat[num]
            mlflow.log_metrics({
                f"{name}_acc": prec[j],
                f"{name}_recall": recall[j],
                f"{name}_f1": f1[j],
            }, step=step)
        cnf_mtrx = confusion_matrix(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(list(num2cat)))
        conf_mtrx_file_path = os.path.join("temp", f"train_cnf_mtrx_{epoch}_{i}.png")
        plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
        mlflow.log_artifact(conf_mtrx_file_path)
        mlflow.log_metric('train_acc', train_acc, step=step)

        if (i+1) % 10 == 0:
            j, data = next(enumerate(test_loader, 0))
            points, labels = data
            points = points.transpose(2, 1)
            points, labels = points.to(device), labels.to(device)
            classifier = classifier.eval()
            with torch.no_grad():
                pred, _ = classifier(points)
            pred = pred.view(-1, num_classes)
            loss = F.nll_loss(pred, labels)
            pred_choice = pred.data.max(1)[1]
            correct = pred_choice.eq(labels.data).cpu().sum()
            test_acc = correct.item() / float(batchsize)
            print(blue('epoch %d: %d/%d | test loss: %f | test acc: %f') % (epoch+1, i+1, num_batch+1, loss.item(), test_acc))
            test_acc_epoch.append(test_acc)
            
            # log test
            cnf_mtrx = confusion_matrix(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(list(num2cat)))
            conf_mtrx_file_path = os.path.join("temp", f"test_cnf_mtrx_{epoch}_{i}.png")
            plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
            mlflow.log_artifact(conf_mtrx_file_path)
            mlflow.log_metric('test_acc', np.mean(test_acc_epoch))
            
        # to next batch
        step += 1
    
    # to next epoch
    print(yellow('epoch %d | mean train acc: %f') % (epoch+1, np.mean(train_acc_epoch)))
    print(red('epoch %d | mean test acc: %f') % (epoch+1, np.mean(test_acc_epoch)))
    torch.save(classifier.state_dict(), '%s/%s_model_%d.pth' % ('models', 'model_10', epoch))

epoch 1: 1/125 | train loss: 2.454874 | train acc: 0.031250
epoch 1: 2/125 | train loss: 2.453289 | train acc: 0.093750
epoch 1: 3/125 | train loss: 1.969043 | train acc: 0.250000
epoch 1: 4/125 | train loss: 2.060935 | train acc: 0.250000
epoch 1: 5/125 | train loss: 1.517412 | train acc: 0.562500
epoch 1: 6/125 | train loss: 1.800226 | train acc: 0.500000
epoch 1: 7/125 | train loss: 2.053360 | train acc: 0.375000
epoch 1: 8/125 | train loss: 1.998027 | train acc: 0.343750
epoch 1: 9/125 | train loss: 1.827359 | train acc: 0.375000
epoch 1: 10/125 | train loss: 1.960671 | train acc: 0.468750
epoch 1: 10/125 | test loss: 18.070076 | test acc: 0.000000
epoch 1: 11/125 | train loss: 1.342815 | train acc: 0.500000
epoch 1: 12/125 | train loss: 1.126386 | train acc: 0.531250
epoch 1: 13/125 | train loss: 1.429792 | train acc: 0.375000
epoch 1: 14/125 | train loss: 1.612843 | train acc: 0.562500
epoch 1: 15/125 | train loss: 1.285018 | train acc: 0.593750
epoch 1: 16/125 | train loss: 1.37

In [55]:
num2class = dict(zip(range(10), train_loader.dataset.raw_file_names))

In [68]:
pointss = data.pos.numpy()
for btch in np.unique(data.batch.numpy()):
    points = pointss[data.batch == btch]
    print(num2class[data.y[btch].numpy().item()])
    print(np.min(points[:, 0], axis=0), np.max(points[:, 0], axis=0), np.max(points[:, 0], axis=0) - np.min(points[:, 0], axis=0))
    print(np.min(points[:, 1], axis=0), np.max(points[:, 1], axis=0), np.max(points[:, 1], axis=0) - np.min(points[:, 1], axis=0))
    print(np.min(points[:, 2], axis=0), np.max(points[:, 2], axis=0), np.max(points[:, 2], axis=0) - np.min(points[:, 2], axis=0))
    print()

bed
-0.8752633 0.89645064 1.771714
-0.99999905 0.89762324 1.8976223
-0.231328 0.4611699 0.6924979

desk
-0.25317165 0.8512118 1.1043835
-0.95747113 0.999999 1.9574702
-0.6959119 0.12586308 0.82177496

sofa
-0.71917844 0.6585192 1.3776977
-0.8147365 0.99999905 1.8147355
-0.1762224 0.4483075 0.6245299

chair
-0.6121519 0.58003956 1.1921915
-0.8185277 0.99923277 1.8177605
-0.56900716 0.8437649 1.412772

table
-0.2510807 0.5558331 0.8069138
-0.36220503 0.36191383 0.7241188
-0.999999 0.10629373 1.1062927

night_stand
-0.10616867 0.7600212 0.8661899
-0.79955983 0.79955995 1.5991198
-0.999999 0.9989007 1.9988997

monitor
-0.3327983 0.999999 1.3327973
-0.66639864 0.66639864 1.3327973
-0.5120062 0.8207911 1.3327973

monitor
-0.1636657 0.39862558 0.56229126
-0.6964982 0.67877895 1.3752772
-0.28227845 0.999999 1.2822775

sofa
-0.92549175 0.36221954 1.2877113
-0.46057156 0.9548646 1.4154361
-0.2584274 0.10073888 0.3591663

sofa
-0.5206773 0.2951557 0.81583303
-0.99939686 0.99364084 1.9930377
-0.48

In [71]:
for data in train_loader:
    pass
    break
    
print(data.y.numpy())
points = data.pos.numpy()[data.batch == 0]
view(points)

[1 7 1 7 7 8 1 7 2 1 6 7 2 4 9 2 5 9 6 8 7 2 8 9 1 5 9 3 2 7 5 7]
